# L2HMC using eager execution in tensorflow

### Imports

In [33]:
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from l2hmc_eager import dynamics_eager as _l2hmc
from l2hmc_eager import gauge_dynamics_eager as l2hmc
from l2hmc_eager.neural_nets import *
from utils.distributions import GMM, gen_ring
from utils.jacobian import _map, jacobian

%autoreload 2

In [34]:
from lattice.ising_lattice import IsingLattice
from lattice.gauge_lattice import GaugeLattice

In [3]:
tf.enable_eager_execution()
tfe = tf.contrib.eager

In [4]:
tfe = tf.contrib.eager

In [5]:
def train_one_iter(dynamics, beta, x, optimizer, 
                   loss_fn=l2hmc.compute_loss, global_step=None):
    loss, grads, out, accept_prob = l2hmc.loss_and_grads(
        dynamics, x, loss_fn=loss_fn
    )
    optimizer.apply_gradients(
        zip(grads, dynamics.trainable_variables), global_step=global_step
    )
    return loss, out, accept_prob

### Construct GaugeLattice with $U(1)$ gauge group

$$ U_{\mu\nu} = \frac{\beta}{3}\sum_{\nu \neq \mu} \mathrm{Re}\left\{\mathrm{Tr}\left[U_{\mu}(x)U_{\nu}(x+\hat\mu)U_{\mu}^{\dagger}(x+\hat\nu)U_{\nu}^{\dagger}(x)\right]\right\}$$

In [111]:
time_size, space_size, dim, beta, num_samples = (8, 8, 2, 2., 5)
u1_lattice = GaugeLattice(time_size, space_size, dim, beta,
                          link_type='U1', num_samples=num_samples)
#u1_samples = u1_lattice.get_links_samples(batch_size, link_type='U1')
#u1_samples_flat = [i.flatten() for i in u1_samples]
u1_samples = [sample.flatten() for sample in u1_lattice.samples]
u1_samples_tensor = tf.constant(np.stack(u1_samples), dtype=tf.float32)

In [112]:
u1_energy_fn = u1_lattice.get_energy_function()
u1_dynamics = l2hmc.GaugeDynamics(u1_lattice, 
                                  minus_loglikelihood_fn=u1_energy_fn, 
                                  n_steps=5, eps=0.1)

In [113]:
[u1_lattice._total_action(sample) for sample in u1_samples]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [114]:
_momentum = tf.random_normal(tf.shape(u1_samples))
_potential = np.array(u1_dynamics.potential(u1_samples_tensor))
_kinetic = u1_dynamics.kinetic(_momentum)
_grad_potential = u1_dynamics.grad_potential(u1_samples_tensor)
print(_potential); print('\n')
print(_kinetic); print('\n')
print(_grad_potential)
#print(_kinetic.numpy()); print('\n')
#print(_hamiltonian.numpy()); print('\n')
#print(_grad_potential[0][:10])

[0. 0. 0. 0. 0.]


tf.Tensor([70.33361  74.60285  71.023346 54.76835  65.8804  ], shape=(5,), dtype=float32)


tf.Tensor(
[[-0.24027282  1.2854897   0.86479557 -2.042479   -1.072858    1.5767564
  -1.392227    2.6517372   3.8235795  -2.7269847  -3.850769    3.7472358
   3.2949734  -1.9772063  -1.4272215  -0.9507253  -0.37073377  0.42497745
  -0.75851077  2.0482838  -0.0248848   1.0003105   2.114161   -2.5060775
   0.9431299   0.37437215 -3.1158872  -0.3605098   2.740976    0.19348739
  -1.5282506   0.2945165   0.9515834  -2.7758794  -0.30784613 -3.2265441
  -0.8428939  -2.408535    0.9131788  -1.2075528   0.24325158 -0.5076744
  -1.4716977  -2.1518638  -2.1026158   2.691728    2.6170397  -1.4535623
   0.47495794  2.5088103   1.383845    0.8171191  -0.37514764  0.34937286
  -1.8594716   3.1220233  -0.07655159  3.4418263   3.215173   -1.2450441
  -1.8014154  -1.5462445  -0.9613895   2.0321848  -0.6626574   1.1027702
  -0.3245958   2.811211    3.92447    -1.4884065  -2.3648474  -0.9830307

#### Run L2HMC for $U(1)$ gauge model

In [115]:
log_dirs = os.listdir('../../U1_logs/')
run_nums = [int(i.split('_')[-1]) for i in log_dirs if i.startswith('run')]
run_num = max(run_nums) + 1
log_dir = f'../../U1_logs/run_{run_num}'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
print(log_dir)

../../U1_logs/run_15


In [116]:
#train_dir = f'../../U1_logs/run_{run_num}'
#train_dir = '../U1_logs/run_7/'
global_step = tf.train.get_or_create_global_step()
_ = global_step.assign(1)
train_iters = 50
record_loss_every = 10
save_steps = 10

learning_rate = tf.train.exponential_decay(1e-3, global_step, 1000, 
                                           0.96, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
checkpointer = tf.train.Checkpoint(
    optimizer=optimizer, dynamics=u1_dynamics, global_step=global_step
)
summary_writer = tf.contrib.summary.create_file_writer(log_dir)
loss_fn = l2hmc.compute_loss
samples = u1_samples_tensor

In [119]:
# 7m13s for 64 links with shape (2, 2, 2, 2, 4), batch_size = 2
# 19m53s for 64 links with shape (2, 2, 2, 2, 4), batch_size = 5
total_actions = []
for i in range(1, 101):
    loss, samples, accept_prob = train_one_iter(
        u1_dynamics,
        u1_lattice.beta,
        u1_samples_tensor,
        optimizer,
        loss_fn=loss_fn,
        global_step=global_step
    )
    _total_actions = u1_lattice.total_action(samples)
    total_actions.extend(_total_actions)

    print(_total_actions)
    print("Iteration {}, loss {:.4f}, x_accept {:.4f},"
          " eps {:.4f}, avg_S {:.4f}".format(i, loss.numpy(),
                                             accept_prob.numpy().mean(),
                                             u1_dynamics.eps.numpy(),
                                             np.mean(_total_actions)))
    print('\n')

    if i % record_loss_every == 0:
        with summary_writer.as_default():
            with tf.contrib.summary.always_record_summaries():
                tf.contrib.summary.scalar("Training loss", loss,
                                          step=global_step)

    if i % save_steps == 0:
        saved_path = checkpointer.save(file_prefix=os.path.join(log_dir,
                                                                "ckpt"))
        print(f"Saved checkpoint to: {saved_path}")

[2.146813798462972, 2.0248913092073053, 1.8782520571839996, 2.3844029866158962, 1.8010301511967555]
Iteration 1, loss -32492.0508, x_accept 1.0000, eps 0.1250, avg_S 2.0471


[1.5395472639975196, 2.406833275977988, 1.7151629656309524, 1.903425661032088, 2.0217914065578952]
Iteration 2, loss -31504.9316, x_accept 1.0000, eps 0.1262, avg_S 1.9174


[2.069585886178629, 1.9744905153056607, 2.012265456978639, 1.9272990712488536, 2.1341814679763047]
Iteration 3, loss -28861.8496, x_accept 1.0000, eps 0.1274, avg_S 2.0236


[1.797601819387637, 2.1482476484961808, 2.1473147825163323, 1.8901303023631044, 2.0379433766938746]
Iteration 4, loss -33267.3477, x_accept 0.9763, eps 0.1281, avg_S 2.0042


[1.7148091907729395, 1.7565187802538276, 1.8573893667708035, 2.182746935635805, 2.3433464415138587]
Iteration 5, loss -30345.0000, x_accept 1.0000, eps 0.1288, avg_S 1.9710


[1.9407270342344418, 2.052834769245237, 1.6536658731056377, 1.635629173453708, 1.8104464931238908]
Iteration 6, loss -32765.515

<tf.Tensor: id=234992, shape=(), dtype=bool, numpy=True>

Saved checkpoint to: ../../U1_logs/run_15/ckpt-11
[2.0135042689507827, 1.9263893789266149, 2.0847050026986835, 1.9016635752559523, 0.0]
Iteration 11, loss -31882.1504, x_accept 0.8009, eps 0.1371, avg_S 1.5853


[2.129803404561244, 1.8150079378392547, 2.035539251053706, 2.034189268364571, 2.1213513921757112]
Iteration 12, loss -31709.3320, x_accept 1.0000, eps 0.1390, avg_S 2.0272


[2.0070910481736064, 1.9207998503698036, 2.0061461140867323, 0.0, 2.0341035445817397]
Iteration 13, loss -31128.3477, x_accept 0.8001, eps 0.1410, avg_S 1.5936


[1.9963229957193107, 2.1613079470407683, 2.0464832006364304, 2.0286029565613717, 0.0]
Iteration 14, loss -29141.9160, x_accept 0.8242, eps 0.1429, avg_S 1.6465


[0.0, 2.06673389027128, 1.7142248819582164, 0.0, 0.0]
Iteration 15, loss -14668.3486, x_accept 0.4000, eps 0.1448, avg_S 0.7562


[0.0, 2.052151211530145, 2.030876693490427, 1.9526942702941597, 0.0]
Iteration 16, loss -19407.4648, x_accept 0.4454, eps 0.1468, avg_S 1.2071


[0.0, 0.0, 0.0,

<tf.Tensor: id=234992, shape=(), dtype=bool, numpy=True>

Saved checkpoint to: ../../U1_logs/run_15/ckpt-12
[0.0, 0.0, 0.0, 0.0, 0.0]
Iteration 21, loss 1999.9980, x_accept 0.0000, eps 0.1544, avg_S 0.0000


[0.0, 0.0, 0.0, 0.0, 0.0]
Iteration 22, loss 1999.9980, x_accept 0.0000, eps 0.1556, avg_S 0.0000


[0.0, 0.0, 0.0, 0.0, 0.0]
Iteration 23, loss 1999.9980, x_accept 0.0000, eps 0.1567, avg_S 0.0000


[0.0, 0.0, 0.0, 0.0, 0.0]
Iteration 24, loss 1999.9980, x_accept 0.0000, eps 0.1577, avg_S 0.0000


[0.0, 0.0, 0.0, 0.0, 0.0]
Iteration 25, loss 1999.9980, x_accept 0.0000, eps 0.1587, avg_S 0.0000


[0.0, 0.0, 0.0, 0.0, 0.0]
Iteration 26, loss 1999.9980, x_accept 0.0000, eps 0.1595, avg_S 0.0000




KeyboardInterrupt: 

In [ ]:
plaqs = []
S = []
S_approx = []
for site in _latt.iter_sites():
    for mu in range(_latt.dim):
        for nu in range(_latt.dim):
            if nu > mu:
                plaq =  _latt.plaquette_operator(_latt.links, site, mu, nu)
                _S = 1 - np.cos(plaq)
                S_approx.append(0.5 * plaq ** 2)
                plaqs.append(plaq)
                S.append(_S)

In [ ]:
np.mean(S_approx)

In [ ]:
_latt._total_action(_latt.links, beta)

In [ ]:
len(S)

In [ ]:
sum(S) / len(S)

In [ ]:
np.mean(beta*np.array(S))

In [ ]:
(_latt._total_action(_latt.links, beta) * _latt.num_sites / (-beta)) / _latt.num_links

In [ ]:
total_actions = np.array(total_actions).reshape(train_iters, batch_size)

In [ ]:
print(samples[0].numpy().reshape(u1_lattice.links.shape))

In [ ]:
saved_path = checkpointer.save(file_prefix=os.path.join(train_dir, 
                                                        "ckpt"))
print(f"Saved checkpoint to: {saved_path}")

In [ ]:
import sys
help(sys.stdout.flush)

In [ ]:
loss, samples, accept_prob

### Construct GaugeLattice with SU(3) gauge group

In [ ]:
time_size = 2
space_size = 4
dim = 4
beta = 1.
link_type = 'SU3' 
batch_size = 3
gauge_lattice = GaugeLattice(time_size, space_size, dim, beta, link_type)
# create `num_samples` random samples of GaugeLattice.links
links_samples = gauge_lattice.get_links_samples(batch_size, link_type=link_type)

In [ ]:
gauge_energy_fn = gauge_lattice.get_energy_function()
gauge_dynamics = l2hmc.GaugeDynamics(gauge_lattice, 
                                     minus_loglikelihood_fn=gauge_energy_fn, 
                                     batch_size=3, n_steps=5, eps=0.1)

In [ ]:
gauge_lattice.links.shape

In [ ]:
potential_arr = gauge_dynamics.potential(links_samples, batch_size)

[i.numpy() for i in potential_arr]

In [ ]:
_momentum = tf.random_normal(tf.shape(links_samples))

In [ ]:
gauge_dynamics.kinetic(_momentum).numpy()

In [ ]:
_x = links_samples
#_momentum = tf.random_normal(tf.shape(_x))
_hamiltonian = gauge_dynamics.hamiltonian(_x, _momentum)
_hamiltonian

### Construct IsingLattice

In [ ]:
ising_batch_size = 10
ising_lattice = IsingLattice(3, 4)
ising_samples = [ising_lattice._randomize() for _ in range(ising_batch_size)]

In [ ]:
ising_energy_fn = ising_lattice.get_energy_function()
ising_dynamics = l2hmc.LatticeDynamics(ising_lattice, 
                                       minus_loglikelihood_fn=ising_energy_fn,
                                       batch_size=ising_batch_size, 
                                       n_steps=10, eps=0.1)
#dynamics = l2hmc.LDynamics(latt.sites.shape, minus_loglikelihood_fn=energy_fn, n_steps=10, eps=0.1)

In [ ]:
ising_dynamics.potential(samples, batch_size)

In [ ]:
_iposition = ising_samples
_imomentum = tf.random_normal(tf.shape(_iposition))
_ihamiltonian = dynamics.hamiltonian(_iposition, _imomentum)
_ihamiltonian

In [ ]:
_isample = _iposition[0].reshape(ising_lattice.num_sites)
#dynamics.grad_potential(np.array(_position).reshape(-1, lattice.num_sites))

In [ ]:
grad_pot = dynamics.grad_potential(ising_samples)

In [ ]:
grad_pot

In [ ]:
ising_jacobian = jacobian(dynamics.potential, ising_samples)

In [ ]:
%debug

In [ ]:
grad_fn = tfe.gradients_function(lattice._calc_energy, params=[0])

In [ ]:
_jacobian = jacobian(dynamics.potential, _position)

In [ ]:
%debug

In [ ]:
lattice.calc_energy(_position, batch_size)

In [ ]:
#dynamics.position_fn(momentum, latt.sites.flatten()[:], dynamics)
#dynamics._forward_lf(latt.sites.flatten()[:], momentum, 0)
dynamics._forward_lf(np.array(_position).reshape(-1, lattice.num_sites),
                     np.array(_momentum).reshape(-1, lattice.num_sites), 1)

### GMM Model

In [ ]:
sigmas, distribution = gen_ring(1., var=0.02, nb_mixtures=4)

gmm_potential = distribution.get_energy_function()
gmm_dynamics = _l2hmc.Dynamics(x_dim=2, minus_loglikelihood_fn=gmm_potential,
                               n_steps=25, eps=0.1)

In [ ]:
samples = distribution.get_samples(200)

In [ ]:
_position = samples
_momentum = tf.random_normal(tf.shape(_position))
_hamiltonian = gmm_dynamics.hamiltonian(_position, _momentum)

In [ ]:
grad_pot = gmm_dynamics.grad_potential(_position, _momentum)

In [ ]:
grad_pot.shape